모델 버전2를 써서 

In [13]:
from ultralytics import YOLO
model = YOLO("modelFolder/model_v2.pt")
#사진해상도 1280*720
results = model(source="w640h360/2_Color.png", device="cpu",project="results",name="output")


image 1/1 c:\Team_Project\w640h360\2_Color.png: 384x640 4 boxs, 1 canSnack, 1 energdrink, 2 milks, 687.2ms
Speed: 2.0ms preprocess, 687.2ms inference, 12.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to results\output3


In [10]:
for result in results:
    xy_list = result.masks.xyn  

In [55]:
import numpy as np
import open3d as o3d
pcd = o3d.io.read_point_cloud('w640h360/2.ply')
o3d.visualization.draw_geometries([pcd])

In [48]:
import numpy as np
import math

def yaw_pitch_roll_to_rotation_matrix(yaw, pitch, roll):
    # 각도를 라디안으로 변환
    yaw_rad = math.radians(yaw)
    pitch_rad = math.radians(pitch)
    roll_rad = math.radians(roll)

    # Yaw, Pitch, Roll 회전 행렬 계산
    yaw_matrix = np.array([
        [math.cos(yaw_rad), -math.sin(yaw_rad), 0],
        [math.sin(yaw_rad), math.cos(yaw_rad), 0],
        [0, 0, 1]
    ])

    pitch_matrix = np.array([
        [math.cos(pitch_rad), 0, math.sin(pitch_rad)],
        [0, 1, 0],
        [-math.sin(pitch_rad), 0, math.cos(pitch_rad)]
    ])

    roll_matrix = np.array([
        [1, 0, 0],
        [0, math.cos(roll_rad), -math.sin(roll_rad)],
        [0, math.sin(roll_rad), math.cos(roll_rad)]
    ])

    # 회전 행렬을 순서대로 곱해서 최종 회전 행렬 생성
    rotation_matrix = np.dot(np.dot(roll_matrix, pitch_matrix), yaw_matrix)
    return rotation_matrix

# 예시 각도 (단위: 도)
# yaw_angle = 30
# pitch_angle = 45
# roll_angle = 60
yaw_angle = 0
pitch_angle = 0
roll_angle = 90

# 회전 행렬 생성
rotation_matrix = yaw_pitch_roll_to_rotation_matrix(yaw_angle, pitch_angle, roll_angle)
print(rotation_matrix)

[[          1           0           0]
 [          0  6.1232e-17          -1]
 [          0           1  6.1232e-17]]


In [56]:
import numpy as np
import open3d as o3d
import math

# # read ply file
# pcd = o3d.io.read_point_cloud('w640h360/2.ply')

# 
# rotate = np.identity(3)  
# size = np.ones(3) * 1.35 # trying to create a bounding box below 1 unit
# center = np.zeros(3) 

# 회전 행렬 참고자료 : https://en.wikipedia.org/wiki/Rotation_matrix
yaw_angle = 0  # + : crop 영역 반시계 방향 회전(카메라 시점), - : crop 영역 시계 방향 회전(카메라 시점)
pitch_angle = 0
roll_angle = 0

rotation_matrix = yaw_pitch_roll_to_rotation_matrix(yaw_angle, pitch_angle, roll_angle)
size = [0.72,  # 너비(카메라 기준, 2d width) 1.139 보다 작아지면 우측 잘리기 시작함, 0.01 = 2 pixel 거리
        0.52,  # 높이(카메라 기준, 2d height) 0.849 보다 작아지면 아래쪽 잘리기 시작함, 0.00875 = 2 pixel 거리
        1.500]  # 깊이(카메라 기준 거리, 3d depth) 1.387 보다 작아지면 좌측 아래쪽 바닥 잘리기 시작함
                # 바닥 제거 1.345

center = [-0.025,  # + : crop 영역이 우측으로 이동, - : crop 영역이 좌측으로 이동 (-0.5195 ~ 0.5708), 합 1.0903
          0.0,  # + : crop 영역이 위쪽으로 이동, - : crop 영역이 아래쪽으로 이동 ( -0.4377 ~ 0.4179)
          0.0,] # + : crop 영역이 카메라쪽으로 이동, - : crop 영역이 아래쪽으로 이동
obb = o3d.geometry.OrientedBoundingBox(center, rotation_matrix, size) # or you can use axis aligned bounding box class

print(rotation_matrix)
print(size)
print(center)

crop_pcd0 = pcd.crop(obb)
# print(pcd)
# visualize
o3d.visualization.draw_geometries([crop_pcd0])
# o3d.io.write_point_cloud("cropply/croppedtest2.ply", crop_pcd)


[[          1           0           0]
 [          0           1           0]
 [          0           0           1]]
[0.72, 0.52, 1.5]
[-0.025, 0.0, 0.0]


In [58]:
import numpy as np
import open3d as o3d

# xy_list에서 첫 번째 배열을 사용하여 새로운 배열 생성
new_xyz_list = np.c_[xy_list[1], np.full(xy_list[1].shape[0], -0.10)]
corners = np.array(new_xyz_list)

#PLY파일이 원본 크기의 0.5배이기 때문에 모든 x,y,z좌표에 0.5 곱해서 비율 맞추기
corners *= 0.5
corners[:, 0] *= 0.72  # x 좌표에 0.72를 곱함
corners[:, 1] *= 0.52  # y 좌표에 0.52을 곱함

# corners 배열의 데이터 유형을 float64로 변환
bounding_polygon = corners.astype("float64")

# SelectionPolygonVolume 생성
vol = o3d.visualization.SelectionPolygonVolume()

# 다각형을 어떤 축을 기준으로 정렬할 것인지 지정해야 합니다.
# 여기서는 "Z" 축을 선택합니다. 최대값은 다각형 꼭지점 중 최대 Z 값,
# 최소값은 다각형 꼭지점 중 최소 Z 값으로 설정합니다.
axis_num = 2
vol.orthogonal_axis = "Z"
vol.axis_max = 10.0
vol.axis_min = -10.0

# Z 값은 설정한 vol.axis_max 및 vol.axis_min 값으로 이미 지정되었으므로
# 모든 Z 값을 0으로 설정합니다.
bounding_polygon[:, axis_num] = 0

# np.array를 Vector3dVector로 변환합니다.
vol.bounding_polygon = o3d.utility.Vector3dVector(bounding_polygon)

print("vol = ", vol)
print("vol.orthogonal_axis = ", vol.orthogonal_axis)
print("vol.axis_min = ", vol.axis_min)
print("vol.axis_max = ", vol.axis_max)
print("vol.bounding_polygon = \n", np.asarray(vol.bounding_polygon))

# Vector3dVector를 사용하여 포인트 클라우드를 자릅니다.
crop_pcd = vol.crop_point_cloud(crop_pcd0)

# 정육면체의 선 정보 (꼭지점을 연결하는 라인 인덱스)
lines = list()
for i in range(len(corners)):
    lines.append([i, (i+1) % len(corners)])

print(lines)

# 라인 정보 생성
lineset = o3d.geometry.LineSet()
lineset.points = o3d.utility.Vector3dVector(corners)
lineset.lines = o3d.utility.Vector2iVector(lines)

# 시각화
o3d.visualization.draw_geometries([crop_pcd])


vol =  SelectionPolygonVolume, access its members:
orthogonal_axis, bounding_polygon, axis_min, axis_max
vol.orthogonal_axis =  Z
vol.axis_min =  -10.0
vol.axis_max =  10.0
vol.bounding_polygon = 
 [[    0.22556     0.14444           0]
 [      0.225     0.14517           0]
 [      0.225     0.14589           0]
 [    0.22275     0.14878           0]
 [    0.22275      0.1495           0]
 [    0.22106     0.15167           0]
 [    0.22106     0.15239           0]
 [     0.2205     0.15311           0]
 [     0.2205     0.15383           0]
 [    0.21825     0.15672           0]
 [    0.21825     0.15744           0]
 [    0.21769     0.15817           0]
 [    0.21769     0.15889           0]
 [    0.21656     0.16033           0]
 [    0.21656     0.16106           0]
 [      0.216     0.16178           0]
 [      0.216      0.1625           0]
 [    0.21544     0.16322           0]
 [    0.21544     0.16394           0]
 [    0.21488     0.16467           0]
 [    0.21431     0.16